In [13]:
!date

Mon Mar 14 19:34:27 CET 2022


In [1]:
import fixed2float
from fixed2float import Fx, to_Fx

fixed2float.__version__

'2.0.4'

In [7]:
LUT_IN = 8
LUT_OUT = 9

N, ES = 16, 1

In [8]:
# let's say we have this posit

from hardposit import from_bits

a = from_bits(0b0111000000101000, N, ES)
a

P<16,1>:         0b0111000000101000   0x7028   28712
                   0111000000101000   
sign:              0
regime:            0000000000001110 -> (reg_s, reg_len) = (1, 4) -> k = 2
exponent:                         0
mantissa:          0000000000101000 -> 10000101000000 Fx<1,14>(8512) ⇒ 1.0390625
                   0111000000101000

                   (-1) ** 0 * (2 ** (2 ** 1)) ** 2 * (2 ** 0) * (1 + 40/1024)
                   16.625
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#### Beware

The following does not consider the only special case
where the fractional bits are zero
that corresponds to a mantissa value of 1.0 whose reciprocate is 1.0,
which does not fit the format Fx<0, LUT_OUT>

In [9]:
fp_mant_a = Fx(a.mant_repr().unwrap(), 1, N-2)

fp_mant_a, fp_mant_a.eval(), fp_mant_a.val

(10000101000000, 1.0390625, 8512)

In [12]:
# now we trim the mantissa to N bits and remove the "1." which provides to additional information

fp_frac_a_N_bits = to_Fx(
    x=fp_mant_a.eval() - 1,
    m=0,
    b=LUT_IN,
    round=True,
)

fp_frac_a_N_bits, fp_frac_a_N_bits.eval(), fp_frac_a_N_bits.val

(00001010, 0.0390625, 10)

In [14]:
# so the value that actually enters the LUT is:

fp_frac_a_N_bits.val

10

In [15]:
fp_mant_a, fp_mant_a.eval()

(10000101000000, 1.0390625)

In [18]:
fp_mant_a_recip1 = to_Fx(
    x=1.0 / fp_mant_a.eval(), 
    m=1, 
    b=1 + LUT_OUT, 
    round=True,
)

fp_mant_a_recip1

0111101101...

In [20]:
# most precise vs (1+LUT_OUT)-bits precise

1.0 / fp_mant_a.eval(), fp_mant_a_recip1.eval()

(0.9624060150375939, 0.962890625)

In [24]:
to_Fx(fp_mant_a_recip1.eval(), 0, LUT_OUT, round=True)

111101101

In [25]:
_.val

493